In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
import re
import camelot

c:\Users\TCL0011\AppData\Local\anaconda3\envs\ml1\Lib\site-packages\pypdf\_crypt_providers\_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4


ImportError: cannot import name 'TableList' from 'camelot.core' (c:\Users\TCL0011\AppData\Local\anaconda3\envs\ml1\Lib\site-packages\camelot\core\__init__.py)

In [5]:
fixture_23 = 'https://res.hkjc.com/racingnews/wp-content/uploads/sites/3/2023/07/fixture_23-24_en.pdf'
fixture_24 = 'https://res.hkjc.com/racingnews/wp-content/uploads/sites/3/2024/07/fixture_24-25_e.pdf'
fixture_25 = 'https://res.hkjc.com/racingnews/wp-content/uploads/sites/3/2025/07/fixture_25-26.pdf'

In [6]:
out_dir = '../data/fixtures'
os.makedirs(out_dir, exist_ok=True)

In [9]:
response_23 = requests.get(fixture_23, stream = True, verify = False)
if response_23.status_code == 200:
    with open((out_dir + '/fixture_23.pdf'), 'wb') as f:
        f.write(response_23.content)
    
    print(f'Downloaded to {out_dir}')

c:\Users\TCL0011\AppData\Local\anaconda3\envs\ml1\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'res.hkjc.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Downloaded to ../data/fixtures


In [11]:
tables_23 = camelot.read_pdf('../data/fixtures/fixture_23.pdf', pages = 'all', flavor = 'stream')

AttributeError: module 'camelot' has no attribute 'read_pdf'

# scrape by date

In [11]:
date = ['2025/07/16']
venue = ['HV', 'ST']

In [12]:
params = {
    'dates': date,
    'venues': venue
}

In [ ]:
def fetch_race_data_by_venue(date, venues, max_race_no=9):
 
    base_url = "https://racing.hkjc.com/racing/information/English/Racing/LocalResults.aspx"
    results = {}

    headers = {
        "User-Agent": "Mozilla/5.0 (compatible; HKJCScraper/1.0)"
    }

    for venue in venues:
        print(f"Checking venue {venue} race no 1...")
        params = {
            "RaceDate": date,
            "Racecourse": venue,
            "RaceNo": 1
        }

        response = requests.get(base_url, params=params, headers=headers)
        if response.status_code != 200:
            print(f"Failed to fetch page for venue {venue} race 1, status: {response.status_code}")
            continue

        soup = BeautifulSoup(response.text, "html.parser")
        # Check for the presence of the race result table, often with class 'table_bd' or id
        table = soup.find('table', class_='table_bd')

        if not table:
            print(f"No race data found for venue {venue} race 1; skipping rest of races for this venue.")
            continue  # Skip race 2-9 for this venue

        # Save race 1 result
        results[(venue, 1)] = soup
        print(f"Race data found for venue {venue} race 1; fetching races 2 to {max_race_no}...")

        # Now fetch the rest of the races for this venue
        for race_no in range(2, max_race_no + 1):
            params["RaceNo"] = race_no
            response = requests.get(base_url, params=params, headers=headers)
            if response.status_code != 200:
                print(f"Failed to fetch page for venue {venue} race {race_no}, status: {response.status_code}")
                results[(venue, race_no)] = None
                continue

            soup = BeautifulSoup(response.text, "html.parser")
            table = soup.find('table', class_='table_bd')
            if not table:
                print(f"No race data found for venue {venue} race {race_no}.")
                results[(venue, race_no)] = None
            else:
                results[(venue, race_no)] = soup

    return results

# Example of calling the function
if __name__ == "__main__":
    race_date = '2025/07/16'
    venue_list = ['HV', 'ST']

    data = fetch_race_data_by_venue(race_date, venue_list)
    # 'data' contains BeautifulSoup objects for successful pages or None entries for misses
